# Python で気軽に化学・化学工学
# 第 8 章 8.5 オーバーフィッティング・多重共線性・部分的最小二乗 (PLS) 法による線形重回帰分析

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### 沸点のデータセット (descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの特徴量 (記述子) で化学構造が数値化されています。記述子は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
dataset.shape

In [ ]:
dataset # 念のため確認

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を 説明変数 x とします

In [ ]:
x # 念のための確認

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

In [ ]:
y # 念のため確認

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=True, random_state=99)

下はテキスト化していますが、shuffle=False とすると、ランダムに分割されるのではなく、下から test_size の数のサンプルがテストデータに、残りのサンプルがトレーニングデータになります。時系列データにおいて、時間的に古いサンプルをトレーニングデータに、新しいサンプルをテストデータとしたいときなどに利用します。

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=False)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

最小二乗 (Ordinary Least Squares, OLS) 法による線形重回帰分析

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # モデルの構築。OLS による線形重回帰分析では、標準回帰係数を計算することに対応します

標準回帰係数

In [ ]:
model.coef_ # 標準回帰係数。array 型で出力されます

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.index = x_train.columns # 変数に対応する名前を、元のデータの変数名に

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients # 念のため確認

例えば HeavyAtomMolWt (水素原子以外で計算された分子量) の標準回帰係数が負に大きい値となっています。水素原子以外で計算された分子量の大きな化合物は沸点が低く推定されるということです。分子量の大きな化合物は沸点が高い傾向がある、という知見と一致しません。このような現象が起きた原因の一つとして、共線性・多重共線性が考えられます。特徴量間の相関係数を見てみましょう。

相関係数の確認

In [ ]:
correlation_coefficients = autoscaled_x_train.corr() #相関行列の計算

In [ ]:
correlation_coefficients # 相関行列の確認

相関行列をヒートマップとして確認

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['font.size'] = 12 # 横軸や縦軸の名前の文字などのフォントのサイズ
sns.heatmap(correlation_coefficients, vmax=1, vmin=-1, cmap='seismic', square=True, xticklabels=1, yticklabels=1)
plt.xlim([0, correlation_coefficients.shape[0]])
plt.ylim([0, correlation_coefficients.shape[0]])
plt.show()

下のようにすれば、ヒートマップの各領域に相関係数の値を入れられます。

In [ ]:
plt.rcParams['font.size'] = 12 # 横軸や縦軸の名前の文字などのフォントのサイズ
sns.heatmap(correlation_coefficients, vmax=1, vmin=-1, cmap='seismic', annot=True, square=True, xticklabels=1, yticklabels=1)
plt.xlim([0, correlation_coefficients.shape[0]])
plt.ylim([0, correlation_coefficients.shape[0]])
plt.show()

HeavyAtomMolWt と相関の非常に高い記述子に、(当たり前ですが) MolWt や NumValenceElectrons や HeavyAtomCount があることがわかります。このように、データセットにおける二つ以上の説明変数間の高い相関関係を多重共線性といい、標準回帰係数の値が不適切に正や負に大きくしてしまう原因の一つです。

PLS による回帰分析の実行

In [ ]:
from sklearn.cross_decomposition import PLSRegression # PLS モデル構築やモデルを用いた y の値の推定に使用

In [ ]:
number_of_components = 2  # 主成分の数

In [ ]:
model = PLSRegression(n_components=number_of_components)

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築

標準回帰係数

In [ ]:
model.coef_ # 標準回帰係数。array 型で出力されます

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.index = x_train.columns # 説明変数に対応する名前を、元のデータの説明変数名に

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients # 念のため確認

例えば 主成分の数が 2 のとき、MolWt (分子量) や HeavyAtomMolWt (水素原子以外で計算された分子量) をはじめとして、すべての標準回帰係数が正の値になっています。大きい分子ほど、各記述子の値も大きくなる傾向があることから、大きな分子の化合物は沸点が高い傾向がある、という知見と一致します。

In [ ]:
standard_regression_coefficients.to_csv('standard_regression_coefficients.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の値の推定

In [ ]:
model.predict(autoscaled_x_train) #トレーニングデータの y の値を推定。array 型で出力されます

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
model.predict(autoscaled_x_test) # テストデータの y の値を推定。array 型で出力されます

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

主成分の数を変えて、それぞれの PLS モデルの標準回帰係数や推定性能を評価してみましょう。

OLS と PLS とで、回帰モデルの標準回帰係数や推定性能を比較してみましょう。

## OLS でオーバーフィッティングが起こり、PLS でそれが軽減される例

### 医薬品錠剤の NIR スペクトルのデータセット (shootout_2002.csv)
2002 年にInternational Diffuse Reflectance Conference (IDRC) が公開した[錠剤の NIR スペクトルのデータセット](http://www.idrc-chambersburg.org/shootout_2002.htm)。460 個の錠剤について、目的変数は錠剤中の有効成分 (Active Pharmaceutical Ingredient, API) の含量 [mg] であり、説明変数は波長 600, 602, ..., 1898 nm で計測された NIR (Near-InfraRed) スペクトル (FOSS NIRSystems Multitab Analysers) 650変数です。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('shootout_2002.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
dataset.shape  # データセットのサンプル数、特徴量の数の確認

In [ ]:
dataset # 念のため確認

In [ ]:
x = dataset.iloc[:, 1:] # スペクトルの特徴量を説明変数 x とします

In [ ]:
x # 念のための確認

In [ ]:
y = dataset.iloc[:, 0] # APIを目的変数 y とします

In [ ]:
y # 念のため確認

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。今回はテストデータのサンプル数を 115 とします
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=115, shuffle=True, random_state=99)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

最小二乗 (Ordinary Least Squares, OLS) 法による線形重回帰分析

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # モデルの構築。OLS による線形重回帰分析では、標準回帰係数を計算することに対応します

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.index = x_train.columns # 変数に対応する名前を、元のデータの変数名に

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients # 念のため確認

In [ ]:
standard_regression_coefficients.to_csv('standard_regression_coefficients_ols.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train_ols.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

実測値 vs. 推定値プロットではサンプルは対角線上にあり、r<sup>2</sup> は 1、MAE はほとんど 0 です。x によって y を完璧に説明できていますが、API の測定結果やスペクトルの測定結果にはノイズが含まれているはずであり、そのような結果は不自然と考えられます。テストデータの y の値を推定して確認してみます。

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # 推定して、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test_ols.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

実測値 vs. 推定値のプロットや MAE の値から、トレーニングデータにおける推定誤差と比較して、テストデータにおける推定誤差が大きいことが確認できます。OLS モデルがトレーニングんデータにオーバーフィッティングしていると考えられます。その原因を検討するため、説明変数の間の相関係数を確認します。

In [ ]:
correlation_coefficients = autoscaled_x_train.corr() #相関行列の計算

In [ ]:
correlation_coefficients # 相関行列の確認

相関行列をヒートマップとして確認 (プロットに少し時間がかかります)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['font.size'] = 12 # 横軸や縦軸の名前の文字などのフォントのサイズ
sns.heatmap(correlation_coefficients, vmax=1, vmin=-1, cmap='seismic', square=True, xticklabels=1, yticklabels=1)
plt.xlim([0, correlation_coefficients.shape[0]])
plt.ylim([0, correlation_coefficients.shape[0]])
plt.show()

説明変数の数が多いため、横軸と縦軸の目盛りは見えにくいですが、ヒートマップから判断すると、特に波長の近い特徴量の間に、とても高い相関関係 (相関係数が 0.99 を超える相関関係) があることがわかります。このように、特徴量の間に強い共線性があると、OLS による線形重回帰分析ではモデルがオーバフィッティングしてしまい、新しいサンプルに対する予測精度が低くなることが多いです。
  
次に、PLS による線形重回帰分析を行います。

PLS による回帰分析の実行

In [ ]:
from sklearn.cross_decomposition import PLSRegression # PLS モデル構築やモデルを用いた y の値の推定に使用

In [ ]:
number_of_components = 5  # 主成分の数

今回は主成分の数をとりあえず 5 としています。最適な主成分数の決め方は 8.6 節で扱います。

In [ ]:
model = PLSRegression(n_components=number_of_components)

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築

標準回帰係数

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.index = x_train.columns # 変数に対応する名前を、元のデータの変数名に

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients # 念のため確認

OLS における標準回帰係数の絶対値と比較して、絶対値が小さい傾向があります。PLS モデルがオーバーフィッティングしている可能性が、OLS モデルと比較して低いと考えられます。

In [ ]:
standard_regression_coefficients.to_csv('standard_regression_coefficients_pls.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # 推定して、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train_pls.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

OLS におけるトレーニングデータの推定結果と比較すると、r<sup>2</sup> は小さく、MAEは大きいです。また実測値 vs. 推定値のプロットより、対角線から離れているサンプルも見られます。次に、テストデータで検証します。

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # 推定して、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test_pls.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

テストデータの推定結果を確認すると、OLS と比べて PLS のほうが r<sup>2</sup> は大きく、MAE は小さく、さらに実測値 vs. 推定値のプロットより対角線付近に固まっていることから、OLS より PLS のほうがテストデータの推定結果は高いといえます。PLS では OLS と比較してオーバーフィッティングが軽減されていることが確認できました。今回のデータセットにおいては、回帰モデル構築に用いていないサンプルに対する推定性能の高い PLS のほうが OLS より望ましいです。

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。

前回の仮想的な樹脂材料のデータセット用いた練習問題をぜひ PLS でも行ってみましょう。